In [114]:
import pandas as pd 
import numpy as np

In [115]:
path='./data/credit-g.csv'
credit_df=pd.read_csv(path)
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   checking_status         1000 non-null   object
 1   duration                1000 non-null   int64 
 2   credit_history          1000 non-null   object
 3   purpose                 1000 non-null   object
 4   credit_amount           1000 non-null   int64 
 5   savings_status          1000 non-null   object
 6   employment              1000 non-null   object
 7   installment_commitment  1000 non-null   int64 
 8   personal_status         1000 non-null   object
 9   other_parties           1000 non-null   object
 10  residence_since         1000 non-null   int64 
 11  property_magnitude      1000 non-null   object
 12  age                     1000 non-null   int64 
 13   other_payment_plans    1000 non-null   object
 14  housing                 1000 non-null   object
 15  exist

In [116]:
credit_df.head()


,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,critical/other existing credit,radio/tv,1169,no known savings,>=7,4,male single,none,...,real estate,67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,existing paid,radio/tv,5951,<100,1<=X<4,2,female div/dep/mar,none,...,real estate,22,none,own,1,skilled,1,none,yes,bad
2,no checking,12,critical/other existing credit,education,2096,<100,4<=X<7,2,male single,none,...,real estate,49,none,own,1,unskilled resident,2,none,yes,good
3,<0,42,existing paid,furniture/equipment,7882,<100,4<=X<7,2,male single,guarantor,...,life insurance,45,none,for free,1,skilled,2,none,yes,good
4,<0,24,delayed previously,new car,4870,<100,1<=X<4,3,male single,none,...,no known property,53,none,for free,2,skilled,2,none,yes,bad


In [117]:
def map_the_column(df,column,mapper):
    return df[column].map(mapper)

column='checking_status'
checking_status_map={
    'no checking': 0,
    '<0': 1,
    '0<=X<200': 2,
    '>=200': 3
}
credit_df['checking_status']=map_the_column(credit_df,column,checking_status_map)

In [118]:
credit_df.columns

Index(['checking_status', 'duration', 'credit_history', 'purpose',
       'credit_amount', 'savings_status', 'employment',
       'installment_commitment', 'personal_status', 'other_parties',
       'residence_since', 'property_magnitude', 'age', ' other_payment_plans',
       'housing', 'existing_credits', 'job', 'num_dependents', 'own_telephone',
       ' foreign_worker', 'class'],
      dtype='object')

In [119]:
# credit_df.dropna(inplace=True,axis=0)

In [120]:
credit_history_mapper = {
    'critical/other existing credit': -3,
    'existing paid': 1,
    'delayed previously': -1,
    'no credits/all paid': 3,
    'all paid': 2
}
column='credit_history'
credit_df['credit_history']=map_the_column(credit_df,column,credit_history_mapper)

In [121]:
credit_df['purpose'].unique()

array(['radio/tv', 'education', 'furniture/equipment', 'new car',
       'used car', 'business', 'domestic appliance', 'repairs', 'other',
       'retraining'], dtype=object)

In [122]:
len(credit_df['purpose'].unique())

10

In [123]:
from sklearn.preprocessing import LabelEncoder
lable_encoder=LabelEncoder()
credit_df['purpose'],_=pd.factorize(credit_df['purpose'])
credit_df['purpose']=lable_encoder.fit_transform(credit_df['purpose'])


In [124]:
credit_df['savings_status'].unique()
savings_status_mapper={
    'no known savings':0, 
    '<100':1, 
    '100<=X<500':2,
    '500<=X<1000':3, 
    '>=1000':4, 
}
column='savings_status'
credit_df['savings_status']=map_the_column(credit_df,column,savings_status_mapper)

In [125]:
credit_df['employment'].unique()
employment_map={
    '>=7':4, 
    '4<=X<7':3, 
    '1<=X<4':2, 
    '<1':1,
    'unemployed':-1 
}
column='employment'
credit_df['employment']=map_the_column(credit_df,column,employment_map)

In [126]:
from sklearn.preprocessing import LabelEncoder
lable_encoder=LabelEncoder()
column='personal_status'
credit_df[column],_=pd.factorize(credit_df[column])
credit_df[column]=lable_encoder.fit_transform(credit_df[column])


In [127]:
credit_df['other_parties'].unique()
column='other_parties'
other_parties_mapper={
    'none':0, 
    'guarantor':1, 
    'co applicant':2
}
credit_df['other_parties']=map_the_column(credit_df,column,other_parties_mapper)


In [128]:
column='property_magnitude'
property_magnitude_mapper={
    'real estate':1, 
    'life insurance':1, 
    'no known property':1, 
    'car':0
    }
credit_df[column]=map_the_column(credit_df,column,property_magnitude_mapper)


In [129]:
credit_df['housing'].unique()
column='housing'
housing_mapper={
 'own':3, 
 'for free':2, 
 'rent' :1  
}
credit_df[column]=map_the_column(credit_df,column,housing_mapper)

In [130]:
credit_df['job'].unique()
column='job'
job_mapper={
 'skilled':1, 
 'unskilled resident':-1, 
 'high qualif/self emp/mgmt':2,
       'unemp/unskilled non res':-2  
}
credit_df[column]=map_the_column(credit_df,column,job_mapper)

In [131]:
credit_df[' foreign_worker'].unique()
from sklearn.preprocessing import LabelEncoder
lable_encoder=LabelEncoder()
column=' foreign_worker'
credit_df[column],_=pd.factorize(credit_df[column])
credit_df[column]=lable_encoder.fit_transform(credit_df[column])

In [132]:
credit_df

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,1,6,-3,0,1169,0,4,4,0,0,...,1,67,none,3,2,1,1,yes,0,good
1,2,48,1,0,5951,1,2,2,1,0,...,1,22,none,3,1,1,1,none,0,bad
2,0,12,-3,1,2096,1,3,2,0,0,...,1,49,none,3,1,-1,2,none,0,good
3,1,42,1,2,7882,1,3,2,0,1,...,1,45,none,2,1,1,2,none,0,good
4,1,24,-1,3,4870,1,2,3,0,0,...,1,53,none,2,2,1,2,none,0,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,12,1,2,1736,1,3,3,1,0,...,1,31,none,3,1,-1,1,none,0,good
996,1,30,1,4,3857,1,2,4,2,0,...,1,40,none,3,1,2,1,yes,0,good
997,0,12,1,0,804,1,4,4,0,0,...,0,38,none,3,1,1,1,none,0,good
998,1,45,1,0,1845,1,2,4,0,0,...,1,23,none,2,1,1,1,yes,0,bad


In [133]:
x===1

SyntaxError: invalid syntax (Temp/ipykernel_46524/2784139477.py, line 1)

In [134]:
from sklearn.model_selection import train_test_split

# credit_df.dropna(inplace=True)
X=credit_df.drop(['class','own_telephone',' other_payment_plans','own_telephone'],axis=1)
y=credit_df['class']


In [135]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=42)


In [136]:
X_train

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,housing,existing_credits,job,num_dependents,foreign_worker
541,0,24,-1,3,2032,1,4,4,0,0,4,1,60,2,2,1,1,0
440,0,12,1,3,1884,1,4,4,0,0,4,0,39,3,1,2,1,0
482,1,30,1,2,3622,4,4,4,1,0,4,1,57,1,2,1,1,0
422,2,12,-3,3,958,1,3,2,0,0,3,1,47,3,2,-1,2,0
778,0,36,-3,4,5711,4,4,4,0,0,2,0,38,3,2,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0,18,2,3,6458,1,4,2,0,0,4,1,39,3,2,2,2,0
270,0,18,1,3,2662,0,3,4,0,0,3,1,32,3,1,1,1,1
860,0,24,-3,4,5804,4,2,4,0,0,2,1,27,3,2,1,1,0
435,2,12,1,0,1484,0,2,2,3,0,1,1,25,3,1,1,1,0


In [137]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
# X_train.dropna(inplace=True)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy with RandomForest: ',accuracy)

from sklearn.linear_model import LogisticRegression


# Logistic Regression with Lasso regularization (L1)
lasso_model = LogisticRegression(penalty='l1', solver='liblinear')
lasso_model.fit(X_train, y_train)
y_pred=lasso_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy with LR and Lasso: ',accuracy)

# Logistic Regression with Ridge regularization (L2)
ridge_model = LogisticRegression(penalty='l2')
ridge_model.fit(X_train, y_train)
y_pred=ridge_model.predict(X_test)
print('Accuracy with LR and Ridge: ',accuracy)

Accuracy with RandomForest:  0.7666666666666667
Accuracy with LR and Lasso:  0.71
Accuracy with LR and Ridge:  0.71


c:\Users\Eniac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
